In [1]:
import torch
import abmap

In [3]:
# Load foundation models (default beplerberger)
abmap.reload_models_to_device(plm_type='beplerberger') # Now allows for cpu, callable from root

cpu


In [3]:
# Load AbMAP 
abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_H_beplerberger_epoch50.pt', plm_name='beplerberger')
abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_L_beplerberger_epoch50.pt', plm_name='beplerberger')

<All keys matched successfully>

In [4]:
# Get embedding for one sequence
demo_seq = 'EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS'

# Contrastive augmentation (PLM, mutagenesis, CDR focus)
x = abmap.ProteinEmbedding(demo_seq, chain_type='H', embed_device=torch.device('cpu'))
x.create_cdr_specific_embedding(embed_type='beplerberger', k=50)

# Pass the augmented embedding to AbMAP to get final embedding
with torch.no_grad():
    embed_var = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='variable')
    embed_fl = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='fixed')

data/anarci_files/temp0
Sequence: EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS
scheme: chothia


tensor([[ 0.4400, -0.0200,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
        [-0.0200,  0.0000, -0.0200,  ...,  0.0000,  0.0000,  1.0000],
        [-0.0200, -0.0600,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
        ...,
        [-0.0200, -0.0400, -0.0600,  ...,  0.0000,  1.0000,  1.0000],
        [-0.0200, -0.0400,  0.0000,  ...,  0.0000,  1.0000,  1.0000],
        [-0.0600, -0.0200, -0.0400,  ...,  0.0000,  1.0000,  1.0000]])

In [18]:
embed_var.shape, embed_fl.shape

(torch.Size([1, 31, 256]), torch.Size([1, 512]))

In [ ]:
# Get embeddings for a fasta file of sequences